## Notebook para testes

In [5]:
import base64

import pandas as pd
import os
import requests
import streamlit as st
from PIL import Image

import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


In [6]:
def get_df_from_csv(csv_name):
    return pd.read_csv(download_csv_file(csv_name), index_col=0)


def download_csv_file(csv_name):
    req = requests.get(
        f"https://raw.githubusercontent.com/pythonbrasil/dados/main/dados/python-brasil-2020/{csv_name}.csv"
    )
    
    if not os.path.exists('./files'):
        os.makedirs('./files')    
    csv_name = f"./files/{csv_name}.csv"
    url_content = req.content
    with open(csv_name, "wb") as csv_file:
        csv_file.write(url_content)
    csv_file.close()
    return csv_name

In [15]:
def get_event_tickets():
    df = get_df_from_csv("inscrições-tutoriais")
    rename_columns = {
        "Orientação sexual:":"Orientação sexual",
        "Se outro, qual?": "Se define - Se outro, qual?",
        "Se outro, qual?.1": "Se identifica - Se outro, qual?",
        "Se outro, qual?.2": "Orientação sexual - Se outro, qual?",
        "Se sim, qual?": "Necessidades específicas - Se sim, qual?",
    }

    return df[
['Como você se define?',
       'Se outro, qual?', 'Como você se identifica?', 'Se outro, qual?.1',
       'Faz parte da população T (pessoa transgênera, travesti)? ',
       'Orientação sexual:', 'Se outro, qual?.2',
       'Pessoa com necessidades específicas?', 'Se sim, qual?']
    ].rename(columns=rename_columns).fillna('N/A')


In [16]:
sample_df = get_event_tickets()
sample_df

,Como você se define?,"Se define - Se outro, qual?",Como você se identifica?,"Se identifica - Se outro, qual?","Faz parte da população T (pessoa transgênera, travesti)?",Orientação sexual,"Orientação sexual - Se outro, qual?",Pessoa com necessidades específicas?,"Necessidades específicas - Se sim, qual?"
0,Uma pessoa branca,SEM RESPOSTA,Mulher,SEM RESPOSTA,Não,Bisexual,SEM RESPOSTA,Não,SEM RESPOSTA
1,Uma pessoa branca,SEM RESPOSTA,Homem,SEM RESPOSTA,Não,Hétero,SEM RESPOSTA,Não,SEM RESPOSTA
2,Uma pessoa branca,SEM RESPOSTA,Homem,SEM RESPOSTA,Não,Hétero,SEM RESPOSTA,Não,SEM RESPOSTA
3,Uma pessoa branca,SEM RESPOSTA,Homem,SEM RESPOSTA,Não,Hétero,SEM RESPOSTA,Não,SEM RESPOSTA
4,Uma pessoa branca,SEM RESPOSTA,Homem,SEM RESPOSTA,Não,Hétero,SEM RESPOSTA,Não,SEM RESPOSTA
...,...,...,...,...,...,...,...,...,...
582,Uma pessoa parda,SEM RESPOSTA,Homem,SEM RESPOSTA,Não,Hétero,SEM RESPOSTA,Não,SEM RESPOSTA
583,Uma pessoa parda,SEM RESPOSTA,Homem,SEM RESPOSTA,Não,Hétero,SEM RESPOSTA,Não,SEM RESPOSTA
584,Uma pessoa amarela,SEM RESPOSTA,Mulher,SEM RESPOSTA,Não,Pansexual,SEM RESPOSTA,Não,SEM RESPOSTA
585,Uma pessoa branca,SEM RESPOSTA,Homem,SEM RESPOSTA,Não,Hétero,SEM RESPOSTA,Não,SEM RESPOSTA


In [19]:
columns_who = ['Como você se define', 'Se define - Se outro, qual?',
       'Como você se identifica?', 'Se identifica - Se outro, qual?',
       'Faz parte da população T (pessoa transgênera, travesti)?',
       'Orientação sexual', 'Orientação sexual - Se outro, qual?',
       'Pessoa com necessidades específicas?',
       'Necessidades específicas - Se sim, qual?']

columns_where = ['Em qual UF você reside?','Em qual país você reside?']

columns_python = ['De quais edições da Python Brasil você já participou?',
       'Você já participou de algum outro evento Python?',
       'Você participou de outros eventos online durante a pandemia de covid-19?',
       'Há quanto tempo você programa em Python?',
       'Como você classificaria seu nível de conhecimento em Python?',
       'Você é estudante?', 'Você trabalha com Python?',
       'Você pretende submeter alguma atividades (palestra, tutorial) para a Python Brasil?',
       'Você faz parte de alguma comunidade local? (grupy, PUG, PyLadies, AfroPython)',
       'comunidade local - Se sim, qual?']

In [37]:
sample_df.shape[0]

7080

In [32]:
for itens in list(sample_df.columns[0:2]):

    categorical_column = itens

    values = sample_df[categorical_column].value_counts().tolist()
    values_perc = [round(value/ sum(values)*100,2) for value in values]
    categories = sample_df[categorical_column].unique().tolist()

    fig = make_subplots(rows=1, cols=1,subplot_titles=("Valores Absolutos", "Percentual") )

    fig.add_trace(
        go.Bar( x=values,y=categories,name=f"{categorical_column}", marker_color="black",orientation='h'),
        row=1, col=1
    )

    fig.update_layout(title_text=f"Quantidade em Absoluto de Respostas: - {categorical_column}")

    fig.show()